In [1]:
import pandas as pd
import numpy as np
import uuid

In [2]:
SUM_INTERVALS = [(-7,0), (-14,0), (-30,0)]

## Generate some random data

In [5]:
uids=np.array([str(uuid.uuid4()) for _ in range(10)])
times = pd.date_range('2016-01-01','2016-02-01', freq='s')
data = dict(
    id=np.random.choice(uids, 100),
    timestamp=np.random.choice(times, 100),
    feature_a=np.ones(100),
    feature_b=np.zeros(100),
)

## Challenge:
The `SUM_INTERVAL` variable contains relative time intervals in days. So the first means 7 days back until today (asof time writing this 2016-02-01).
the dictionary `data` contains 10 distinct users which made visists over the timespan of one month. Each visit has a value for `feature a`, as awell as `feature_b` assigned to it. **For each user calculate the sum of it's respective features for each time interval. The final output should be a dataframe or numpy matrix containing one row per user, an id column an the feature columns (N_users, 1 + N_intervals*N_features)**

*It is encouraged to use the pandas library for this task but it is not required.*

In [6]:
def bin_sum_features(data, today=pd.Timestamp('2016-02-01')):
    pass

## Result shape example
below you see how the results could look like with pandas

In [7]:
pd.DataFrame([['1aa9204b-5956-41a3-96b6-58cbf6bc147e',1,2,3,4,5,6]], 
             columns=['id', 'feature_a_7', 'feature_a_14', 'feature_a_30', 
                      'feature_b_7', 'feature_b_14', 'feature_b_30'])

,id,feature_a_7,feature_a_14,feature_a_30,feature_b_7,feature_b_14,feature_b_30
0,1aa9204b-5956-41a3-96b6-58cbf6bc147e,1,2,3,4,5,6
